### 2. **Пагинация и отбор данных**

#### **Настройка пагинации с учётом фильтров и сортировки**

Пагинация реализуется с помощью класса `Paginator`, позволяя делить данные на страницы и эффективно отображать большие объёмы информации. 

##### **1. Основы работы с `Paginator`**

Для начала создаём файл `views.py` (если он уже существует, добавляем в него код):

**Файл: `views.py` в папке приложения (например, `myapp/views.py`)**


In [ ]:
from django.core.paginator import Paginator
from .models import Order

def order_list_view(request):
    orders = Order.objects.all()
    paginator = Paginator(orders, 10)  # 10 записей на странице
    page_number = request.GET.get('page', 1)
    page_obj = paginator.get_page(page_number)

    return render(request, 'orders/order_list.html', {'page_obj': page_obj})

Здесь:
   - `Paginator` создаёт страницы на основе `orders` с 10 записями на каждой странице.
   - `get_page(page_number)` возвращает нужные записи на основе параметра `page` из `GET`-запроса.

##### **2. Пагинация с фильтрацией и сортировкой**

Для фильтрации добавим запрос к объектам на основе временных интервалов и сортировку. Например, отобразим заказы, сделанные за последний месяц, отсортированные по дате создания:

In [ ]:
from django.utils import timezone
from datetime import timedelta
from django.core.paginator import Paginator

def order_list_view(request):
    one_month_ago = timezone.now() - timedelta(days=30)
    filtered_orders = Order.objects.filter(created_at__gte=one_month_ago).order_by('-created_at')

    paginator = Paginator(filtered_orders, 10)
    page_number = request.GET.get('page')
    page_obj = paginator.get_page(page_number)

    return render(request, 'orders/order_list.html', {'page_obj': page_obj})

##### **3. Эффективная обработка больших объёмов данных**

При работе с большим объёмом данных важно оптимизировать запросы:
   - Используйте `.only()` или `.defer()` для ограничения полей, загружаемых из базы данных.
   - Применяйте индексы к полям, по которым выполняется частая фильтрация (например, `created_at`).


#### **Интерактивная фильтрация и сортировка**

Чтобы добавить фильтрацию и сортировку на основе `GET`-параметров, создадим форму для передачи фильтров и используем AJAX для асинхронной подгрузки данных.

##### **1. Настройка фильтрации и сортировки через `GET`-параметры**

Для этого в `views.py` добавим обработку параметров `start_date`, `end_date` и `sort`:


In [ ]:
from django.shortcuts import render
from django.utils import timezone
from datetime import timedelta
from .models import Order

def order_list_view(request):
    start_date = request.GET.get('start_date')
    end_date = request.GET.get('end_date')
    sort = request.GET.get('sort', '-created_at')

    orders = Order.objects.all()
    if start_date:
        orders = orders.filter(created_at__gte=start_date)
    if end_date:
        orders = orders.filter(created_at__lte=end_date)

    orders = orders.order_by(sort)

    paginator = Paginator(orders, 10)
    page_number = request.GET.get('page')
    page_obj = paginator.get_page(page_number)

    return render(request, 'orders/order_list.html', {'page_obj': page_obj})

##### **2. Шаблон `order_list.html`**

**Файл: `templates/orders/order_list.html`**

Создаём форму фильтрации в шаблоне и добавляем пагинацию.

```html
<form id="filter-form" method="GET">
    <input type="date" name="start_date" placeholder="Start Date">
    <input type="date" name="end_date" placeholder="End Date">
    <select name="sort">
        <option value="-created_at">Newest First</option>
        <option value="created_at">Oldest First</option>
    </select>
    <button type="submit">Apply Filters</button>
</form>

<div id="order-list">
    {% for order in page_obj %}
        <div>{{ order }}</div>
    {% endfor %}
</div>

<div class="pagination">
    {% if page_obj.has_previous %}
        <a href="?page={{ page_obj.previous_page_number }}">Previous</a>
    {% endif %}

    {% if page_obj.has_next %}
        <a href="?page={{ page_obj.next_page_number }}">Next</a>
    {% endif %}
</div>
```

##### **3. Асинхронная подгрузка данных с фильтрацией и сортировкой**

Добавим AJAX для обновления списка заказов без перезагрузки страницы. В `order_list.html` добавим JavaScript код для отправки AJAX-запроса:


```html
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
<script>
    $('#filter-form').on('submit', function(e) {
        e.preventDefault();
        $.ajax({
            url: $(this).attr('action'),
            type: 'GET',
            data: $(this).serialize(),
            success: function(response) {
                $('#order-list').html(response.html);
            }
        });
    });
</script>
```

##### **4. Частичный шаблон для AJAX**

Создадим шаблон, который будет возвращать HTML для подгрузки в AJAX-запросе:

**Файл: `templates/orders/order_list_partial.html`**

```html
{% for order in page_obj %}
    <div>{{ order }}</div>
{% endfor %}

<div class="pagination">
    {% if page_obj.has_previous %}
        <a href="?page={{ page_obj.previous_page_number }}">Previous</a>
    {% endif %}

    {% if page_obj.has_next %}
        <a href="?page={{ page_obj.next_page_number }}">Next</a>
    {% endif %}
</div>
```

##### **5. Изменения в `views.py` для AJAX**

Изменим `views.py` для возврата данных в формате JSON при AJAX-запросах:


In [ ]:
from django.template.loader import render_to_string
from django.http import JsonResponse

def order_list_view(request):
    orders = Order.objects.all()
    start_date = request.GET.get('start_date')
    end_date = request.GET.get('end_date')
    sort = request.GET.get('sort', '-created_at')
    
    if start_date:
        orders = orders.filter(created_at__gte=start_date)
    if end_date:
        orders = orders.filter(created_at__lte=end_date)
    orders = orders.order_by(sort)

    paginator = Paginator(orders, 10)
    page_number = request.GET.get('page')
    page_obj = paginator.get_page(page_number)

    if request.is_ajax():
        html = render_to_string('orders/order_list_partial.html', {'page_obj': page_obj})
        return JsonResponse({'html': html})

    return render(request, 'orders/order_list.html', {'page_obj': page_obj})

Эти шаги позволяют создать систему, которая обрабатывает пагинацию, фильтрацию и сортировку данных как на серверной стороне, так и интерактивно на клиентской стороне с использованием AJAX.